(EBs with 100% confidence value from Yifan's RF algorithm)
<br>
Finding and removing closest stars, then printing the remaining light curves into a pdf file to visually examine

In [1]:
#imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
#sector you want to classify
sector = 1

read in the prediction as well as the confidence values for each tic in sector 1, based off the RF model

In [3]:
sec1 = pd.read_csv('Sector1/sect1predandprob.csv', index_col=0)

In [ ]:
!gcsfuse --implicit-dirs tess-goddard-lcs /home/jupyter/mountpoint

In [4]:
data_dir = "/home/jupyter/mountpoint/"
ref = pd.read_csv(data_dir + 'sector1lookup.csv')
eb = sec1[sec1['probone'] == 1]

In [5]:
eb

,probzero,probone,pred
140608211,0.0,1.0,1
231791552,0.0,1.0,1
300740040,0.0,1.0,1
181052118,0.0,1.0,1
38811805,0.0,1.0,1
...,...,...,...
260950407,0.0,1.0,1
349641813,0.0,1.0,1
33734375,0.0,1.0,1
141122198,0.0,1.0,1


In [6]:
tics = eb.index.values
len(tics)

337

In [8]:
stars = ref.TIC_ID.isin(tics)
closestarslist = []
for i in range(len(tics)):
    rastar = ref[stars].RA.values[i]
    decstar = ref[stars].Dec.values[i]
    dist = np.sqrt(((rastar-ref[stars].RA.values)*(np.cos(decstar)))**2+(decstar-ref[stars].Dec.values)**2)
    closestars = np.where((dist < 0.1) & (dist != 0))
    closestarslist.append(closestars)
    
closestarstics = []
for i in range (len(closestarslist)):
    tics = []
    tics.append(ref[stars].TIC_ID.values[i])
    for j in range(len(closestarslist[i][0])):
        tics.append(ref[stars].TIC_ID.values[closestarslist[i][0][j]])
    closestarstics.append(tics)

In [10]:
closestarslist

[(array([ 78, 145]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([165]),),
 (array([], dtype=int64),),
 (array([100]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([252, 253, 259, 264]),),
 (array([183]),),
 (array([], dtype=int64),),
 (array([193]),),
 (array([279]),),
 (array([ 31, 254, 334]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([305]),),
 (array([], dtype=int64),),
 (array([336]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([335]),),
 (array([198, 292, 293]),),
 (array([ 19, 254, 334]),),
 (array([119, 120]),),
 (array([], dtype=int64),),
 (array([143]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([211, 212]),),
 (array(

In [11]:
closestarstics

[[139699256, 2026930048, 139699260],
 [139841210],
 [144462697],
 [204497617],
 [25226885],
 [141268467],
 [33862174],
 [389476160],
 [261746657],
 [33715938, 33715940],
 [303909865],
 [201256771, 201256776],
 [91369561],
 [44623916],
 [126945917, 126945904, 126945919, 126945918, 126945913],
 [410354930, 410354938],
 [55652785],
 [147201138, 147201134],
 [89018905, 2027112725],
 [260161144, 737425239, 260161142, 737425238],
 [219338651],
 [220388567],
 [270204401, 318606776],
 [350443417],
 [350622185, 734511288],
 [167304040],
 [393948907],
 [314887863],
 [114948643],
 [370239670, 370239669],
 [197979841, 2026841336, 197980089, 197979845],
 [737425239, 260161144, 260161142, 737425238],
 [389020348, 389020349, 389020350],
 [270174474],
 [300654002, 300657354],
 [409136916],
 [382101519],
 [327937353],
 [309472288, 309549613, 309472204],
 [167651110],
 [309781192],
 [214367709],
 [206363145],
 [140761918],
 [231269538, 161167795],
 [33734375, 33734376],
 [300034498],
 [300091984],
 [234

In [12]:
refebs = ref.loc[ref['TIC_ID'].isin(eb.index.values)]
files = data_dir + refebs.Filename.values
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [13]:
len(lcs)

337

In [14]:
tic = []
avgflux = []
    
for i in range(len(lcs)):
    tic.append(refebs['TIC_ID'].values[i])
    avgflux.append(np.median(lcs[i].flux.value))

In [15]:
df = {'TIC': tic, 'avgflux': avgflux}
avg = pd.DataFrame(df)

In [16]:
highestfluxtics = []
for i in range(len(closestarstics)):
    a = []
    for j in closestarstics[i]:
        a.append(avg[avg['TIC'] == j].avgflux.values[0])
    highestfluxtics.append(closestarstics[i][a.index(max(a))])

In [17]:
len(highestfluxtics)

337

In [18]:
def checkIfDuplicates_2(listOfElems):
    ''' Check if given list contains any duplicates '''    
    setOfElems = set()
    for elem in listOfElems:
        if elem in setOfElems:
            return True
        else:
            setOfElems.add(elem)         
    return False

In [19]:
result = checkIfDuplicates_2(highestfluxtics)
if result:
    print('Yes, list contains duplicates')
else:
    print('No duplicates found in list')   

Yes, list contains duplicates


In [20]:
newlist = [] #eb list with no duplicates, cut down to ones with highest flux values in a radius of 0.1
duplist = []
for i in highestfluxtics:
    if i not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)

In [21]:
len(newlist)

287